Jenna Ruzekowicz (jenna.ruzekowicz@nrel.gov) and Caleb Phillips (caleb.phillips@nrel.gov)

The purpose of this notebook is to read in two sets of data over the same time period for comparison:
1) WTK-LED data
2) Either: 
    a) power output and/or wind speed data from turbine(s) 
    b) wind speed measurements from met tower(s)

Data sets are matched on location and time stamp.

A combined and labeled dataframe will be exported to a csv file in the "01Data" folder. The naming convention for the csv file will be as follows: source_lat_lon_startdate_enddate.csv
where source is either "bergey", "oneenergy"...

Notes: 
Might need to install Rex if it isn't installed already:
conda install nrel-rex --channel=nrel

More about rex: https://github.com/NREL/rex
2018 5-min monthly h5 (the file you referenced on the 21st):
/campaign/tap/CONUS/wtk/5min/2018/{month}/conus_2018-{month}.h5
 
2018 5-min yearly h5 slices:
/shared-projects/wtk-led/CONUS/wtk/2018/yearly_h5/conus_2018_{height}m.h5
 
2019 60-min yearly h5:
/campaign/tap/CONUS/wtk/60min/2019/conus_2019.h5

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from rex.resource_extraction import MultiYearWindX
from dw_tap.data_fetching import get_data_wtk_led_on_eagle